In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [2]:
all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]


## 三種買賣策略

In [3]:
def strategy4_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.32:
                profit = 0.32
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.08:
                profit = -0.08
                break
            elif profit > 0.16:
                profit1 = 0.16
    return profit
def strategy5_profit(stock:pd.DataFrame, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    profit1 = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit1 != 0:
            if profit > 0.40:
                profit = 0.40
                return (profit+profit1)/2
            elif profit <=0:
                profit = 0
                return (profit+profit1)/2
        else:
            if profit <-0.06:
                profit = -0.06
                break
            elif profit > 0.20:
                profit1 = 0.20
    return profit
# def strategy5_profit(stock, day, day1M):
#     initial_price = stock.loc[stock.index>day, 'Adj Close']
#     close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
#     low_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Low']
#     initial_price = close_price.iloc[0]
#     profit1 = 0
#     last_low = low_price.iloc[0]
#     for i in range(len(close_price)-1):
#         profit = (close_price.iloc[i+1]-initial_price)/initial_price
#         if close_price.iloc[i+1]<last_low:
#             if profit1 != 0:
#                 return (profit1+profit)/2
#             else:
#                 return profit
#         if profit >=  0.17:
#             profit1 = 0.17
#         last_low = low_price.iloc[i]
#     if profit1 != 0:
#         return (profit1+profit)/2
#     else:
#         return profit
def strategy6_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    last_profit = 0
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if all([last_profit<0,profit<0]):
            return profit
        if profit <-0.06:
            profit = -0.06
            return profit
        last_profit = profit
    return profit
# stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{"2330"}.TW.csv', parse_dates=['Date'], index_col=['Date'])
# print(strategy5_profit(stock, day, day1M))

In [4]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5006, -36, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #TM
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            # allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            # allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            #T5
            allstock.loc[id, 'RS 20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 250rate>55'] = (allstock.loc[id, 'RS 250rate'] > 55)
            allstock.loc[id, 'RS 250rate<75'] = (allstock.loc[id, 'RS 250rate'] < 75)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #, 'year low sort', 'year high sort'
            #T6-
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            #T11
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate'] > 75)
            #,'RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k'
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'T5'] = all(allstock.loc[id, ['RS 20rate>85', 'RS 250rate>55', 'RS 250rate<75', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T6'] = all(allstock.loc[id, ['RS EMA250rate>80', 'RS EMA20rate>80', 'RS EMA20rate<99', 'Volume 50MA>250k']])
            allstock.loc[id, 'T11'] = all(allstock.loc[id, ['RS EMA250rate>75','RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k']])
            allstock.loc[id, 'TM'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
    T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=True).iloc[-50:].index.values.astype(str)
    T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
    TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
    
    T5_stock_num = len(T5_ID)
    T6_stock_num = len(T6_ID)
    T11_stock_num = len(T11_ID)
    TM_stock_num = len(TM_ID)
    print('刪除前  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([T5_ID,T6_ID,T11_ID,TM_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'T5', 'T6', 'T11', 'TM'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['T5','T6','T11','TM']] = False
    apexstock.loc[T5_ID,'T5'] = True
    apexstock.loc[T6_ID,'T6'] = True
    apexstock.loc[T11_ID,'T11'] = True
    apexstock.loc[TM_ID,'TM'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'T5', 'T6', 'T11', 'TM'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock = apexstock.dropna()
    print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            except:
                try:
                    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
                except Exception as e:
                    print(e)
                    try:
                        apexstock.drop(ID, inplace=True)
                        print(f'drop stock : {ID}')
                    except:
                        pass
                    continue
            got_profit_list = ([ID in T5_ID, ID in T6_ID, ID in T11_ID, ID in TM_ID])
            # print(got_profit_list)
            try:
                initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                
                profit1 = round(strategy4_profit(stock, day, day1M),3)
                profit2 = round(strategy5_profit(stock, day, day1M),3) 
                profit3 = round(strategy6_profit(stock, day, day1M),3) 
                profit4 = round((oneM_price-initial_price)/initial_price,3)
                
                profit_list.append([ID, got_profit_list[0]*profit1, got_profit_list[0]*profit2, got_profit_list[0]*profit3, got_profit_list[0]*profit4
                                    , got_profit_list[1]*profit1, got_profit_list[1]*profit2, got_profit_list[1]*profit3, got_profit_list[1]*profit4
                                    , got_profit_list[2]*profit1, got_profit_list[2]*profit2, got_profit_list[2]*profit3, got_profit_list[2]*profit4
                                    , got_profit_list[3]*profit1, got_profit_list[3]*profit2, got_profit_list[3]*profit3, got_profit_list[3]*profit4])
            except Exception as e:
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    T5_stock_num = len(apexstock.loc[apexstock['T5']].index.values)
    T6_stock_num = len(apexstock.loc[apexstock['T6']].index.values)
    T11_stock_num = len(apexstock.loc[apexstock['T11']].index.values)
    TM_stock_num = len(apexstock.loc[apexstock['TM']].index.values)
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['T5 S4 profit', 'T5 S5 profit', 'T5 S6 profit', 'T5 NS profit'
                                                   , 'T6 S4 profit', 'T6 S5 profit', 'T6 S6 profit', 'T6 NS profit'
                                                   , 'T11 S4 profit', 'T11 S5 profit', 'T11 S6 profit', 'T11 NS profit'
                                                   , 'TM S4 profit', 'TM S5 profit', 'TM S6 profit', 'TM NS profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'T5 S4 profit', 'T5 S5 profit', 'T5 S6 profit', 'T5 NS profit'
                                                   , 'T6 S4 profit', 'T6 S5 profit', 'T6 S6 profit', 'T6 NS profit'
                                                   , 'T11 S4 profit', 'T11 S5 profit', 'T11 S6 profit', 'T11 NS profit'
                                                   , 'TM S4 profit', 'TM S5 profit', 'TM S6 profit', 'TM NS profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] \
    = profit_df.loc[apexstock.index.values,profit_column]
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose for all template (S4-6))(1Week)/' + str(day).split(' ')[0] + 'all template選股(S4-6))(1Week)' + '.xlsx', encoding='utf-8-sig')
    profit = {'T5 S4 profit':100*profit_df.loc[apexstock.index.values,'T5 S4 profit'].sum()/T5_stock_num
              ,'T5 S5 profit':100*profit_df.loc[apexstock.index.values,'T5 S5 profit'].sum()/T5_stock_num
              ,'T5 S6 profit':100*profit_df.loc[apexstock.index.values,'T5 S6 profit'].sum()/T5_stock_num
              ,'T5 NS profit':100*profit_df.loc[apexstock.index.values,'T5 NS profit'].sum()/T5_stock_num
              ,'T5 numbers of stock ':T5_stock_num
              ,'T6 S4 profit':100*profit_df.loc[apexstock.index.values,'T6 S4 profit'].sum()/T6_stock_num
              ,'T6 S5 profit':100*profit_df.loc[apexstock.index.values,'T6 S5 profit'].sum()/T6_stock_num
              ,'T6 S6 profit':100*profit_df.loc[apexstock.index.values,'T6 S6 profit'].sum()/T6_stock_num
              ,'T6 NS profit':100*profit_df.loc[apexstock.index.values,'T6 NS profit'].sum()/T6_stock_num
              ,'T6 numbers of stock ':T6_stock_num
              ,'T11 S4 profit':100*profit_df.loc[apexstock.index.values,'T11 S4 profit'].sum()/T11_stock_num
              ,'T11 S5 profit':100*profit_df.loc[apexstock.index.values,'T11 S5 profit'].sum()/T11_stock_num
              ,'T11 S6 profit':100*profit_df.loc[apexstock.index.values,'T11 S6 profit'].sum()/T11_stock_num
              ,'T11 NS profit':100*profit_df.loc[apexstock.index.values,'T11 NS profit'].sum()/T11_stock_num
              ,'T11 numbers of stock ':T11_stock_num
              ,'TM S4 profit':100*profit_df.loc[apexstock.index.values,'TM S4 profit'].sum()/TM_stock_num
              ,'TM S5 profit':100*profit_df.loc[apexstock.index.values,'TM S5 profit'].sum()/TM_stock_num
              ,'TM S6 profit':100*profit_df.loc[apexstock.index.values,'TM S6 profit'].sum()/TM_stock_num
              ,'TM NS profit':100*profit_df.loc[apexstock.index.values,'TM NS profit'].sum()/TM_stock_num
              ,'TM numbers of stock ':TM_stock_num}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    clear_output()
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    print('day', day)
    display(all_df)
    display(all_df.sum())

刪除後  T5 股票數量 :  47 | T6 股票數量 :  50 | T11 股票數量 :  53 | TM 股票數量 :  254 | apexstock shape : (315, 40)
day 2023-06-28 00:00:00


,T5 S4 profit,T5 S5 profit,T5 S6 profit,T5 NS profit,T5 numbers of stock,T6 S4 profit,T6 S5 profit,T6 S6 profit,T6 NS profit,T6 numbers of stock,T11 S4 profit,T11 S5 profit,T11 S6 profit,T11 NS profit,T11 numbers of stock,TM S4 profit,TM S5 profit,TM S6 profit,TM NS profit,TM numbers of stock
2023-06-28,1.417021,1.870213,4.897872,3.451064,47,2.312,3.890,6.522,2.648,50,1.650943,2.037736,1.796226,0.966038,53,2.580315,3.724016,5.940157,4.615354,254
2023-06-21,4.065957,4.212766,0.340426,6.765957,47,1.594,2.242,-2.350,3.042,50,0.770270,1.405405,1.994595,4.470270,37,1.893358,2.675646,0.957934,3.842066,271
2023-06-14,1.502381,1.592857,-1.592857,3.978571,42,5.102,4.362,1.598,6.306,50,1.373810,0.628571,-2.150000,1.211905,42,2.768321,2.726718,1.157634,3.736641,262
2023-06-07,2.125490,2.574510,0.470588,1.776471,51,6.382,5.148,6.804,10.264,50,1.130435,1.867391,2.691304,2.219565,46,3.262731,3.290037,2.645018,4.395572,271
2023-05-31,7.823810,8.845238,8.564286,11.852381,42,5.994,6.352,6.336,9.282,50,2.752941,3.715686,2.154902,3.990196,51,4.330627,4.696310,3.867528,6.340590,271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-23,-0.725000,-1.250000,-0.450000,-3.537500,8,1.328,1.974,2.562,1.380,50,2.745455,3.004545,4.522727,4.013636,22,2.854000,3.144000,2.634000,1.830000,50
2009-12-16,2.609091,4.245455,3.300000,5.500000,11,4.096,4.106,3.538,6.058,50,2.989474,2.802632,2.726316,5.147368,38,7.785455,8.423636,6.109091,10.847273,55
2009-12-09,5.115385,6.500000,4.946154,7.630769,13,1.944,2.572,5.354,6.272,50,5.235556,4.731111,7.677778,9.011111,45,5.980769,6.038462,5.073077,7.153846,52
2009-12-02,12.966667,14.466667,17.175000,19.758333,12,11.160,11.794,8.688,13.712,50,12.592857,13.988095,17.423810,20.642857,42,11.779545,13.365909,14.497727,18.581818,44


In [5]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\all template(S4-6)(1Week).xlsx')